# Combined science observations and telescope tracking and DIMM to focal plane seeing comparison. STARTING AT UT=0200

- At UT=0200 acquire Mu Col rising on the horizon at 20 deg elevation, tell DIMM to point at Mu Col as well. 
- Do a move-the-secondary focus sequence in red filter, no disperser. 
- Set to that value adjust pointing to put Mu Col at (1750,300)

- Iterate through this sequence as Mu Col rises and transits, until sky is too bright in morning twilight: Take 
                        none	sdss g	10 x 3	1 min
                        none	FELH0600	10 x 3	1 min
                        hologram	none	30 x 3	2 min
                        hologram	BG40	30 x 3	2 min
                        hologram	sdss g	30 x 3	2 min
                        hologram 	FELH0600	30 x 3	2 min

                        this cycle should take 10 minutes


- Every 30 minutes:
    - Switch to FELH0600 band, no disperser.
    - Do move-the-mirror focus sequence, set to best focus value. 
    - Adjust centering on hologram disperser 


In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Instantiate the Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f"Your UID is {os.getuid()}")
index = os.getuid() * 10 + np.random.randint(0, 9)
logger.info(f"The generated index is {index}")

Instantiate the script then start all remotes

In [ ]:
script = LatissAcquireAndTakeSequence(index=index)
await script.start_task

Set the script to have a DEBUG log level

In [ ]:
script.log.level = logging.DEBUG

## Slew to a target
This is not part of the script but seems to come in handy, so it is here for convenience

In [ ]:
# Check Elevation!!! Point DIMM to same source and check data is being published. 
target_name = 'HD 37795'
await atcs.slew_object(name=target_name, rot_type=RotType.PhysicalSky)

In [ ]:
# Red filter? 
test_image = await latiss.take_engtest(
    exptime=15, n=1, filter='FELH0600', grating='empty_1')

print ('Check Saturation test_image = ' + str(test_image))

In [ ]:
# Focus in SDSSg --- CWFS. Paste results here for record. 

## Acquire and take science collection script

In [ ]:
async def acquisition(target):
    """
    ?????
    """
    # Be sure to set the target, filter and grating to the appropriate setup 
    # you want to use for data taking. The position of the target on the 
    # detector is related to the grating.
    configuration = yaml.safe_dump({
        "object_name": target,
        "do_acquire": True,
        "acq_filter": 'FELH0600',
        "acq_grating": 'empty_1',
        "acq_exposure_time": 2.0,
        "target_pointing_tolerance": 3.0,  # Target positioning tolerance (arcsec)
        "max_acq_iter": 3,  # Maximum number of iterations to perform when acquiring
        "target_pointing_verification": False,  # Takes an additional image showing the final target position
    })

    # Set script state to UNCONFIGURED
    # this is required to run the script a 2nd time but otherwise is a no-op
    script.set_state(ScriptState.UNCONFIGURED)
    
    # Configure the script, which puts the ScriptState to CONFIGURED
    config_data = script.cmd_configure.DataType()
    config_data.config = configuration
    await script.do_configure(config_data)

    # run script, assigning the same group ID to all the images taken in the 
    # acquisition
    group_id_data = script.cmd_setGroupId.DataType(
        groupId=astropy.time.Time.now().isot)
    
    await script.do_setGroupId(group_id_data)

    await script.arun()

In [ ]:
async def take_science_data_collection(target):
    """
    ???
    """
    configuration = yaml.safe_dump({
        "object_name": target,
        "do_take_sequence": True,
        "exposure_time_sequence" : [10, 10, 30, 30, 30, 30] * 3, 
        "grating_sequence": ['none', 'none', 'holo4_003', 'holo4_003', 
                             'holo4_003','holo4_003'] * 3,
        "filter_sequence": ['SDSSg', 'FELH0600', 'none,', 'BG40', 
                            'SDSSg','FELH0600'] * 3, 
    })
    
    # Set script state to UNCONFIGURED
    # this is required to run the script a 2nd time but otherwise is a no-op
    script.set_state(ScriptState.UNCONFIGURED)
    
    # Configure the script, which puts the ScriptState to CONFIGURED
    config_data = script.cmd_configure.DataType()
    config_data.config = configuration
    await script.do_configure(config_data)
    
    # run script, assigning the same group ID to all the images taken in the acquisition
    group_id_data = script.cmd_setGroupId.DataType(
        groupId=astropy.time.Time.now().isot)
    
    await script.do_setGroupId(group_id_data)

    await script.arun()

## Acquire target

In [ ]:
await acquisition(target)

## Take 3 of each

In [ ]:
for i in range(3): 
    await take_science_data_collection(target)

## Every 30 minutes. Focus on FELH0600

In [ ]:
test_image = await latiss.take_engtest(exptime=15, n=1, filter='FELH0600',grating='empty_1')

In [ ]:
# Run CWFS on FELH0600 and paste results here:


# Adjunt centering on the hologram disperser. 

In [ ]:
# Keep repeating science_data_collection until sky is too bright. 